### step 4

In [1]:
import numpy as np
from random import shuffle
from math import log, floor
import pandas as pd
import tensorflow as tf
import tensorboard as tb
from keras import backend as K
from keras.models import *
from keras.layers import *
from keras.activations import *
from keras.callbacks import *
from keras.utils import *
from keras.layers.advanced_activations import *
from collections import Counter
from keras import *
from keras.engine.topology import *
from keras.optimizers import *
import keras
# import pandas as pd
import glob
from sklearn.semi_supervised import *
import pickle
from keras.applications import *
from keras.preprocessing.image import *
from keras.losses import mse, binary_crossentropy
import pandas as pd # data frame
import numpy as np # matrix math
from scipy.io import wavfile # reading the wavfile
from sklearn.utils import shuffle # shuffling of data
from random import sample # random selection
from tqdm import tqdm # progress bar
import matplotlib.pyplot as plt # to view graphs
import wave
from math import log, floor
# audio processing
from scipy import signal # audio processing
from scipy.fftpack import dct
import librosa # library for audio processing
import numpy as np
import pandas as pd
from sklearn.decomposition import *
from sklearn.cluster import KMeans
import sys, os
import random,math
from tqdm import tqdm ##
from xgboost.sklearn import XGBClassifier
from sklearn.utils import shuffle # shuffling of data
from random import sample # random selection
from tqdm import tqdm # progress bar
# audio processing
from scipy import signal # audio processing
from scipy.fftpack import dct
import librosa # library for audio processing
import xgboost as xgb
import lightgbm as lgb
import catboost as ctb
from keras.utils import *
from sklearn.ensemble import *
import pickle
from bayes_opt import BayesianOptimization
from logHandler import Logger
from utils import readCSV, getPath, writePickle,readPickle
from keras.regularizers import l2
from keras.callbacks import History ,ModelCheckpoint, EarlyStopping

import resnet
from random_eraser import get_random_eraser
from mixup_generator import MixupGenerator

/home/leoqaz12/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
def _shuffle(X, Y):
    randomize = np.arange(len(X))
    np.random.shuffle(randomize)
#     print(X.shape, Y.shape)
    return (X[randomize], Y[randomize])

def getTrainData():
    X = []
    y = []

    for i in range(num_fold):
        fileX = os.path.join(base_data_path, 'X/X' + str(i+1) + '.npy')
        fileY = os.path.join(base_data_path, 'y/y' + str(i+1) + '.npy')
        
        X.append(np.load(fileX))
        y.append(np.load(fileY))

    X = np.array(X)
    y = np.array(y)

    return X, y

def split_data(X, y, idx):
    X_train = []
    y_train = []
    
    for i in range(num_fold):
        if i == idx:
            X_val = X[i]
            y_val = y[i]
            continue
        if X_train == []:
            X_train = X[i]
            y_train = y[i]
        else:
            X_train = np.concatenate((X_train, X[i]))
            y_train = np.concatenate((y_train, y[i]))

    return X_train, y_train, X_val, y_val

def normalize(X_train, X_val):
    X_train = (X_train - mean)/(std)
#     X_train = (X_train - min_)/range_
    X_val = (X_val - mean)/(std)
#     X_val = (X_val - min_)/range_

    return X_train, X_val


In [3]:

# min_ = np.swapaxes(min_,0,1)
# mean = np.swapaxes(mean,0,1)
# range_ = np.swapaxes(range_,0,1)
# std = np.swapaxes(std,0,1)

In [4]:
def train_valid(X_train,Y_train,X_valid,Y_valid,fold):
    model = [resnet.ResnetBuilder.build_resnet_18((1, X_train.shape[1], X_train.shape[2]), 41),
             resnet.ResnetBuilder.build_resnet_34((1,X_train.shape[1], X_train.shape[2]), 41),
             resnet.ResnetBuilder.build_resnet_50((1, X_train.shape[1], X_train.shape[2]), 41) ,]
#              resnet.ResnetBuilder.build_resnet_101((1, X_train.shape[1], X_train.shape[2]), 41),]
#              resnet.ResnetBuilder.build_resnet_152((1, X_train.shape[1], X_train.shape[2]), 41)]
    kk = random.randint(0, 2)#4)
    model = model[kk]
    print('using resnet model: '+str(kk))

    model.summary()
    if kk>=3:
        batchSize=[32]
    elif kk>2:
        batchSize=[32,64]#,128,256]
    elif kk>=1:
        batchSize=[32,64,128]
    else:
        batchSize=[32,64,128,256]
    batchSize = random.choice(batchSize)
    patien=100
    epoch=3000
    saveD = 'model/'+feature_type+'/'
    if not os.path.exists(saveD):
        os.makedirs(saveD)
    opt = Adam()#Nadam() #Adam(lr=2e-3,decay=1e-20)
    
    datagen = ImageDataGenerator(
#         featurewise_center=True,  # set input mean to 0 over the dataset
        width_shift_range=0.1*random.random(),
        height_shift_range=0.1*random.random(),
        shear_range=0.05*random.random(),
        preprocessing_function=get_random_eraser(p=0.82+0.13*random.random(),v_l=np.min(X_train), v_h=np.max(X_train)) # Trainset's boundaries.
    )
#     datagen.fit(X_train)
    generator = MixupGenerator(X_train, Y_train, alpha=1, 
                               batch_size=batchSize, datagen=datagen)

    model.compile(loss=['categorical_crossentropy'],optimizer=opt, metrics=['acc']) 
    logD = './logs/'+feature_type+'/'
    print('using resnet model: '+str(kk))
    if not os.path.exists(logD):
        os.makedirs(logD)
    history = History()
    callback=[
    #     ReduceLROnPlateau(monitor='loss', factor=0.5, patience=int(patien/2),
    #                                   min_lr=1e-4,mode='min', cooldown=1 ),
        EarlyStopping(patience=patien,monitor='val_loss',verbose=1,
                      mode='min'),
        ModelCheckpoint(saveD+'LGD_fold'+str(fold)+'_resnet'+str(kk)+'-.h5',
                        monitor='val_acc',verbose=1,save_best_only=True, 
                        save_weights_only=False,mode='max'),
        TensorBoard(log_dir=logD+'LGD_fold'+str(fold)+'_resnet'+str(kk)),
        history#,batch_size=batch_size, write_graph=True, write_grads=False, write_images=True)
    ]

    model.fit_generator(generator(),
                        steps_per_epoch=X_train.shape[0] // batchSize,
                        shuffle=True,
                        callbacks=callback, 
                        class_weight='auto',
                        validation_data=(X_valid,Y_valid),
                        max_queue_size = 32,
                        workers = 11,
#                         use_multiprocessing = True,
#                         batch_size=batchSize,
                        epochs=epoch
                       )
    return model,kk

In [5]:
def get_semi_data(X_train,Y_train):
    X_un_ver = np.load('feature/'+feature_type+'/semi/fbank4/X_un_ver.npy')
    X_test_ver = np.load('feature/'+feature_type+'/semi/fbank4/X_test_ver.npy')
    X = np.concatenate((X_un_ver,X_test_ver))
    Y_un_ver = np.load('feature/'+feature_type+'/semi/fbank4/Y_un_ver.npy')
    Y_test_ver = np.load('feature/'+feature_type+'/semi/fbank4/Y_test_ver.npy')
    Y = np.concatenate((Y_un_ver,Y_test_ver))
    Y = to_categorical(Y,num_classes=41)
    X_semi = np.concatenate((X_train,X))
    Y_semi = np.concatenate((Y_train,Y))
    X_semi , Y_semi = _shuffle(X_semi,Y_semi)
    print(X_semi.shape , Y_semi.shape)
    return X_semi , Y_semi

In [6]:
# model = resnet.ResnetBuilder.build_resnet_50((1, 64, 431), 41)
# model.summary()
def train_unverified(model,X_semi,Y_semi,fold,kk):
    name = glob.glob('model/'+feature_type+'/'+'LGD_fold'+str(fold)+'_resnet'+str(kk)+'-**')[0]
    print('semi loading: '+ name)
    model = load_model(name)
    if kk==4:
        batchSize=[32]
    elif kk>=2:
        batchSize=[32,64]#,128,256]
    elif kk>=1:
        batchSize=[32,64,128]
    else:
        batchSize=[32,64,128,256]
#     batchSize=[32,64,128,256] ##ERR?
    batchSize = random.choice(batchSize)
    patien=100
    epoch=3000
    saveD = 'model/'+feature_type+'/'
    opt = Adam(lr=0.0001,decay=1e-6)#Nadam() #Adam(lr=2e-3,decay=1e-20)
    
    
    datagen = ImageDataGenerator(
#         featurewise_center=True,  # set input mean to 0 over the dataset
#         featurewise_std_normalization=True,
        width_shift_range=0.05+0.15*random.random(),
        height_shift_range=0.05+0.15*random.random(),
        shear_range=0.084375+0.253125*random.random(),
        preprocessing_function=get_random_eraser(v_l=np.min(X_semi), v_h=np.max(X_semi)) # Trainset's boundaries.
    )
#     datagen.fit(X_semi)
    test_datagen = ImageDataGenerator(featurewise_center=True,featurewise_std_normalization=True)
    generator = MixupGenerator(X_semi, Y_semi, alpha=0.4+0.6*random.random(), 
                               batch_size=batchSize, datagen=datagen)
    

    model.compile(loss=['categorical_crossentropy'],optimizer=opt, metrics=['acc']) 
    logD = './logs/'+feature_type+'/'
    history = History()
    callback=[
        ReduceLROnPlateau(monitor='loss', factor=0.5, patience=int(patien/10),min_lr=4e-6,
                          mode='min', cooldown=1,verbose=1 ), #0.2,/25
        EarlyStopping(patience=patien,monitor='val_loss',verbose=1,
                      mode='min'),
        ModelCheckpoint(saveD+'LGD_fold'+str(fold)+'_semi_resnet'+str(kk)+'.h5',
                        monitor='val_acc',verbose=1,save_best_only=True, 
                        save_weights_only=False,
                        mode='max'),
        TensorBoard(log_dir=logD+'LGD_fold'+str(fold)+'_semi_resnet'+str(kk)),
        history
    ]
    model.fit_generator(generator(),
                        steps_per_epoch=X_semi.shape[0] // batchSize,
                        shuffle=True,
                        callbacks=callback, 
                        class_weight='auto',
                        validation_data=(X_valid,Y_valid),
                        max_queue_size = 32,
                        workers = 11,
#                         use_multiprocessing = True,
#                         batch_size=batchSize,
                        epochs=epoch,
                        initial_epoch = int(patien/25)
                       )
#     model.fit(X_semi,Y_semi,
#               shuffle=True,
#               callbacks=callback, 
#               class_weight='auto',
#               validation_data=(X_valid,Y_valid),
#               batch_size=batchSize,
#               epochs=epoch)
    return model

In [7]:
feature_type = 'mfcc7'

In [8]:
# normalized parameters
mean = np.load('feature/'+feature_type+'/mean.npy')
std = np.load('feature/'+feature_type+'/std.npy')
min_ = np.load('feature/'+feature_type+'/min.npy')
range_ = np.load('feature/'+feature_type+'/range.npy')


base_path = 'feature/'+feature_type+'/'#'/tmp2/b03902110/newphase1'
base_data_path = 'feature/'+feature_type+'/'#os.path.join(base_path, 'data')
num_fold = 10

val_set_num = [7,8,9]#str(sys.argv[1])

In [9]:
# fold=0
for fold in val_set_num:
    X, y = getTrainData()
    # X = np.swapaxes(X,2,3)
    X_train, Y_train, X_valid, Y_valid = split_data(X, y, fold) #fold
    # X_train, X_valid = normalize(X_train, X_valid)
    print(X_train.shape, Y_train.shape)

    # X_train = np.swapaxes(X_train,1,3)
    # X_valid = np.swapaxes(X_valid,1,3)
    print("===train verified_fold"+str(fold)+'_'+feature_type+'===')
    model,model_num = train_valid(X_train,Y_train,X_valid,Y_valid,fold)
    X_semi , Y_semi = get_semi_data(X_train,Y_train)
    print('===train semi_'+str(fold)+'===')
    model_semi = train_unverified(model,X_semi,Y_semi,fold,model_num)


/home/leoqaz12/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:32: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.


(3339, 60, 259, 1) (3339, 41)
===train verified_fold6_mfcc7===
using resnet model: 2
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 60, 259, 1)   0                                            
__________________________________________________________________________________________________
conv2d_57 (Conv2D)              (None, 30, 130, 64)  3200        input_3[0][0]                    
__________________________________________________________________________________________________
batch_normalization_53 (BatchNo (None, 30, 130, 64)  256         conv2d_57[0][0]                  
__________________________________________________________________________________________________
activation_51 (Activation)      (None, 30, 130, 64)  0           batch_normalization_53[0][0]     
________________________

Epoch 1/3000
52/52 [==============================] - 16s 302ms/step - loss: 9.0078 - acc: 0.0604 - val_loss: 8.8012 - val_acc: 0.0647

Epoch 00001: val_acc improved from -inf to 0.06469, saving model to model/mfcc7/LGD_fold6_resnet2-.h5
Epoch 2/3000
52/52 [==============================] - 9s 165ms/step - loss: 7.7923 - acc: 0.0739 - val_loss: 7.1946 - val_acc: 0.0997

Epoch 00002: val_acc improved from 0.06469 to 0.09973, saving model to model/mfcc7/LGD_fold6_resnet2-.h5
Epoch 3/3000
52/52 [==============================] - 9s 165ms/step - loss: 6.8076 - acc: 0.0995 - val_loss: 6.8510 - val_acc: 0.0782

Epoch 00003: val_acc did not improve from 0.09973
Epoch 4/3000
52/52 [==============================] - 9s 166ms/step - loss: 6.0597 - acc: 0.1187 - val_loss: 6.1507 - val_acc: 0.0485

Epoch 00004: val_acc did not improve from 0.09973
Epoch 5/3000
52/52 [==============================] - 9s 166ms/step - loss: 5.5071 - acc: 0.1385 - val_loss: 5.0723 - val_acc: 0.1725

Epoch 00005: val_

52/52 [==============================] - 9s 167ms/step - loss: 2.7563 - acc: 0.4567 - val_loss: 2.3626 - val_acc: 0.4286

Epoch 00042: val_acc did not improve from 0.48248
Epoch 43/3000
52/52 [==============================] - 9s 167ms/step - loss: 2.7926 - acc: 0.4555 - val_loss: 2.2227 - val_acc: 0.4394

Epoch 00043: val_acc did not improve from 0.48248
Epoch 44/3000
52/52 [==============================] - 9s 167ms/step - loss: 2.7858 - acc: 0.4444 - val_loss: 2.5051 - val_acc: 0.3747

Epoch 00044: val_acc did not improve from 0.48248
Epoch 45/3000
52/52 [==============================] - 9s 167ms/step - loss: 2.7206 - acc: 0.4627 - val_loss: 2.0551 - val_acc: 0.5229

Epoch 00045: val_acc improved from 0.48248 to 0.52291, saving model to model/mfcc7/LGD_fold6_resnet2-.h5
Epoch 46/3000
52/52 [==============================] - 9s 167ms/step - loss: 2.7352 - acc: 0.4769 - val_loss: 2.2166 - val_acc: 0.5013

Epoch 00046: val_acc did not improve from 0.52291
Epoch 47/3000
52/52 [========

52/52 [==============================] - 9s 167ms/step - loss: 2.4187 - acc: 0.5691 - val_loss: 1.6502 - val_acc: 0.6334

Epoch 00084: val_acc improved from 0.61995 to 0.63342, saving model to model/mfcc7/LGD_fold6_resnet2-.h5
Epoch 85/3000
52/52 [==============================] - 9s 167ms/step - loss: 2.4170 - acc: 0.5673 - val_loss: 1.7660 - val_acc: 0.5633

Epoch 00085: val_acc did not improve from 0.63342
Epoch 86/3000
52/52 [==============================] - 9s 167ms/step - loss: 2.4438 - acc: 0.5634 - val_loss: 1.8612 - val_acc: 0.5580

Epoch 00086: val_acc did not improve from 0.63342
Epoch 87/3000
52/52 [==============================] - 9s 167ms/step - loss: 2.4751 - acc: 0.5493 - val_loss: 2.3727 - val_acc: 0.4232

Epoch 00087: val_acc did not improve from 0.63342
Epoch 88/3000
52/52 [==============================] - 9s 167ms/step - loss: 2.4273 - acc: 0.5628 - val_loss: 1.9449 - val_acc: 0.5391

Epoch 00088: val_acc did not improve from 0.63342
Epoch 89/3000
52/52 [========

...

53/53 [==============================] - 6s 117ms/step - loss: 1.3221 - acc: 0.8650 - val_loss: 0.7326 - val_acc: 0.9003

Epoch 00308: val_acc did not improve from 0.90566
Epoch 309/3000
53/53 [==============================] - 6s 117ms/step - loss: 1.2890 - acc: 0.8806 - val_loss: 0.7327 - val_acc: 0.9003

Epoch 00309: val_acc did not improve from 0.90566
Epoch 310/3000
53/53 [==============================] - 6s 117ms/step - loss: 1.2897 - acc: 0.8659 - val_loss: 0.7328 - val_acc: 0.9003

Epoch 00310: val_acc did not improve from 0.90566
Epoch 311/3000
53/53 [==============================] - 6s 118ms/step - loss: 1.3001 - acc: 0.8656 - val_loss: 0.7332 - val_acc: 0.9003

Epoch 00311: val_acc did not improve from 0.90566
Epoch 312/3000
53/53 [==============================] - 6s 117ms/step - loss: 1.3035 - acc: 0.8623 - val_loss: 0.7352 - val_acc: 0.9003

Epoch 00312: val_acc did not improve from 0.90566
Epoch 313/3000
53/53 [==============================] - 6s 118ms/step - loss: 1.

53/53 [==============================] - 6s 117ms/step - loss: 1.2935 - acc: 0.8735 - val_loss: 0.7304 - val_acc: 0.8949

Epoch 00352: val_acc did not improve from 0.90566
Epoch 353/3000
53/53 [==============================] - 6s 118ms/step - loss: 1.2987 - acc: 0.8691 - val_loss: 0.7304 - val_acc: 0.8976

Epoch 00353: val_acc did not improve from 0.90566
Epoch 354/3000
53/53 [==============================] - 6s 117ms/step - loss: 1.2936 - acc: 0.8617 - val_loss: 0.7296 - val_acc: 0.8976

Epoch 00354: val_acc did not improve from 0.90566
Epoch 355/3000
53/53 [==============================] - 6s 117ms/step - loss: 1.2881 - acc: 0.8620 - val_loss: 0.7292 - val_acc: 0.8976

Epoch 00355: val_acc did not improve from 0.90566
Epoch 356/3000
53/53 [==============================] - 6s 117ms/step - loss: 1.2885 - acc: 0.8729 - val_loss: 0.7275 - val_acc: 0.8976

Epoch 00356: val_acc did not improve from 0.90566
Epoch 357/3000
53/53 [==============================] - 6s 117ms/step - loss: 1.

53/53 [==============================] - 6s 117ms/step - loss: 1.2876 - acc: 0.8665 - val_loss: 0.7308 - val_acc: 0.9003

Epoch 00396: val_acc did not improve from 0.90566
Epoch 397/3000
53/53 [==============================] - 6s 117ms/step - loss: 1.2779 - acc: 0.8700 - val_loss: 0.7289 - val_acc: 0.8976

Epoch 00397: val_acc did not improve from 0.90566
Epoch 398/3000
53/53 [==============================] - 6s 118ms/step - loss: 1.2639 - acc: 0.8747 - val_loss: 0.7298 - val_acc: 0.8976

Epoch 00398: val_acc did not improve from 0.90566
Epoch 399/3000
53/53 [==============================] - 6s 118ms/step - loss: 1.2938 - acc: 0.8756 - val_loss: 0.7315 - val_acc: 0.8976

Epoch 00399: val_acc did not improve from 0.90566
Epoch 400/3000
53/53 [==============================] - 6s 117ms/step - loss: 1.2798 - acc: 0.8721 - val_loss: 0.7308 - val_acc: 0.8976

Epoch 00400: val_acc did not improve from 0.90566
Epoch 401/3000
53/53 [==============================] - 6s 118ms/step - loss: 1.

53/53 [==============================] - 6s 117ms/step - loss: 1.2708 - acc: 0.8768 - val_loss: 0.7326 - val_acc: 0.9003

Epoch 00440: val_acc did not improve from 0.90566
Epoch 441/3000
53/53 [==============================] - 6s 117ms/step - loss: 1.2832 - acc: 0.8726 - val_loss: 0.7316 - val_acc: 0.9003

Epoch 00441: val_acc did not improve from 0.90566
Epoch 442/3000
53/53 [==============================] - 6s 117ms/step - loss: 1.2966 - acc: 0.8715 - val_loss: 0.7305 - val_acc: 0.9003

Epoch 00442: val_acc did not improve from 0.90566
Epoch 443/3000
53/53 [==============================] - 6s 117ms/step - loss: 1.2741 - acc: 0.8726 - val_loss: 0.7311 - val_acc: 0.9003

Epoch 00443: val_acc did not improve from 0.90566
Epoch 444/3000
53/53 [==============================] - 6s 117ms/step - loss: 1.2872 - acc: 0.8785 - val_loss: 0.7321 - val_acc: 0.9003

Epoch 00444: val_acc did not improve from 0.90566
Epoch 445/3000
53/53 [==============================] - 6s 117ms/step - loss: 1.

53/53 [==============================] - 6s 117ms/step - loss: 1.2837 - acc: 0.8638 - val_loss: 0.7295 - val_acc: 0.9030

Epoch 00484: val_acc did not improve from 0.90566
Epoch 00484: early stopping


In [ ]:
#MFCC7
#0=>0.84367(resnet3_semi)/0.83288(not semi)
#1=>0.84367(resnet3_semi)/0.81132(not semi)
#2=>0.84097(resnet2_semi)/0.81671(not semi)
#3=>0.88410(resnet1_semi)/0.84367
#4=>0.82210(resnet4_semi)/0.74663(not semi)
#5 => 0.82749(resnet4_semi)/0.77628
#6 => 0.85445(resnet2_semi)/0.78437
#7 => 0.82749(resnet3_semi)/0.76280
#8 => 0.83288(resnet3_semi)/0.78706
# 9=> 0.90566(resnet1_semi)/0.87332

In [10]:
#MFCC6
# 0=> 0.81941 (not semi)
# 1=>0.83019 (semi)
# 2=>0.81941 (semi)
# 3=>0.85984 (resnet1_not semi)、0.78437
# 4=>0.84367 (renet1_not semi)、0.81132
# 5=> 0.85175(resnet3_semi)、0.82749(not semi)
#6 => 0.85904(resnet4_semi)、0.79784(not semi)
# 7=>0.88949(resnet2_semi)、0.83558(not semi)
# 8=>0.83558(resnet1 not semi)
# 9=>0.86792(resnet2_semi)、0.8112(not semi)

In [13]:
# np.load('feature/mfcc6/fname_unverified.npy')

array(['00044347.wav', '002d256b.wav', '003b91e8.wav', ...,
       'fff37590.wav', 'fff44ac6.wav', 'fff6a13d.wav'], dtype=object)

### Step 8
# Co-Train

In [2]:
def _shuffle(X, Y):
    randomize = np.arange(len(X))
    np.random.shuffle(randomize)
#     print(X.shape, Y.shape)
    return (X[randomize], Y[randomize])

def getTrainData():
    X = []
    y = []

    for i in range(num_fold):
        fileX = os.path.join(base_data_path, 'X/X' + str(i+1) + '.npy')
        fileY = os.path.join(base_data_path, 'y/y' + str(i+1) + '.npy')
        
        X.append(np.load(fileX))
        y.append(np.load(fileY))

    X = np.array(X)
    y = np.array(y)

    return X, y

def split_data(X, y, idx):
    X_train = []
    y_train = []
    
    for i in range(num_fold):
        if i == idx:
            X_val = X[i]
            y_val = y[i]
            continue
        if X_train == []:
            X_train = X[i]
            y_train = y[i]
        else:
            X_train = np.concatenate((X_train, X[i]))
            y_train = np.concatenate((y_train, y[i]))

    return X_train, y_train, X_val, y_val

def normalize(X_train, X_val):
    X_train = (X_train - mean)/(std)
#     X_train = (X_train - min_)/range_
    X_val = (X_val - mean)/(std)
#     X_val = (X_val - min_)/range_

    return X_train, X_val


In [3]:
# model = resnet.ResnetBuilder.build_resnet_50((1, 64, 431), 41)
# model.summary()
def train_unverified(X_semi,Y_semi,fold):
    name = glob.glob('model/'+feature_type+'/'+'**_fold'+str(fold)+'_**')[0]
#     if fold in [0,1,2,3,4,5]: # MFCC6
#         name = glob.glob('model/'+feature_type+'/'+'**_fold'+str(fold)+'_co_**')[0]
    print('semi loading: '+ name)
    model = load_model(name)
    model.summary()
    if 'resnet4'in name:
        batchSize=[32]
    elif 'resnet2' in name or 'resnet3' in name:
        batchSize=[32,64]#,128,256]
    elif 'resnet1' in name:
        batchSize=[32,64,128]
    else:
        batchSize=[32,64]
#     batchSize=[32,64,128,256] ##ERR?
    batchSize = random.choice(batchSize)
    print('BS:',str(batchSize))
    patien=30
    epoch=3000
    saveD = 'model/'+feature_type+'/'
    opt = Adam(lr=0.0001,decay=1e-6)#Nadam() #Adam(lr=2e-3,decay=1e-20)
    
    
    datagen = ImageDataGenerator(
#         featurewise_center=True,  # set input mean to 0 over the dataset
#         featurewise_std_normalization=True,
        width_shift_range=0.05+0.22*random.random(),
        height_shift_range=0.05+0.22*random.random(),
        shear_range=0.084375+0.187625*random.random(),
        preprocessing_function=get_random_eraser(v_l=np.min(X_semi), v_h=np.max(X_semi)) # Trainset's boundaries.
    )
#     datagen.fit(X_semi)
#     test_datagen = ImageDataGenerator(featurewise_center=True,featurewise_std_normalization=True)
    generator = MixupGenerator(X_semi, Y_semi, alpha=0.4+0.6*random.random(), 
                               batch_size=batchSize, datagen=datagen)
    

    model.compile(loss=['categorical_crossentropy'],optimizer=opt, metrics=['acc']) 
    logD = './logs/'+feature_type+'/'
    history = History()
    callback=[
        ReduceLROnPlateau(monitor='loss', factor=0.5, patience=int(patien/4),min_lr=4e-6,
                          mode='min', cooldown=1,verbose=1 ), #0.2,/25 #0.5/5/3e-6 #0.3/10/1e-6
        EarlyStopping(patience=patien,monitor='val_loss',verbose=1,
                      mode='min'),
        ModelCheckpoint(saveD+'LGD_fold'+str(fold)+'_self_resnet'+'.h5',
                        monitor='val_acc',verbose=1,save_best_only=True, 
                        save_weights_only=False,
                        mode='max'),
        TensorBoard(log_dir=logD+'LGD_fold'+str(fold)+'_self_resnet'),
        history
    ]
    model.fit_generator(generator(),
                        steps_per_epoch=2*X_semi.shape[0] // batchSize,
                        shuffle=True,
                        callbacks=callback, 
                        class_weight='auto',
                        validation_data=(X_valid,Y_valid),
                        max_queue_size = 32,
                        workers = 11,
#                         use_multiprocessing = True,
#                         batch_size=batchSize,
                        epochs=epoch,
#                         initial_epoch = int(patien/20)
                       )
#     model.fit(X_semi,Y_semi,
#               shuffle=True,
#               callbacks=callback, 
#               class_weight='auto',
#               validation_data=(X_valid,Y_valid),
#               batch_size=batchSize,
#               epochs=epoch)
    return model

In [4]:
def get_semi_data(X_train,Y_train):
    X = np.load('feature/'+feature_type+'/semi/'+who+'/X_co.npy')
#     X_test_ver = np.load('feature/'+feature_type+'/semi/'+who+'/X_test_ver.npy')
#     X = np.concatenate((X_un_ver,X_test_ver))
    Y = np.load('feature/'+feature_type+'/semi/'+who+'/Y_co.npy')
#     Y_test_ver = np.load('feature/'+feature_type+'/semi/'+who+'/Y_test_ver.npy')
#     Y = np.concatenate((Y_un_ver,Y_test_ver))
    Y = to_categorical(Y,num_classes=41)
    X_semi = np.concatenate((X_train,X))
    Y_semi = np.concatenate((Y_train,Y))
    X_semi , Y_semi = _shuffle(X_semi,Y_semi)
    print(X_semi.shape , Y_semi.shape)
    return X_semi , Y_semi

In [5]:
feature_type = 'mfcc7'
who = 'Y_selftrain_ens_verified'

In [6]:
base_path = 'feature/'+feature_type+'/'#'/tmp2/b03902110/newphase1'
base_data_path = 'feature/'+feature_type+'/'#os.path.join(base_path, 'data')
num_fold = 10

val_set_num = [4,8,9,5]#str(sys.argv[1])

In [7]:
for fold in val_set_num:
    X, y = getTrainData()
    # X = np.swapaxes(X,2,3)
    X_train, Y_train, X_valid, Y_valid = split_data(X, y, fold) #fold
    # X_train, X_valid = normalize(X_train, X_valid)
    print(X_train.shape, Y_train.shape)

    # X_train = np.swapaxes(X_train,1,3)
    # X_valid = np.swapaxes(X_valid,1,3)
#     print("===train verified_fold"+str(fold)+'_'+feature_type+'===')
#     model,model_num = train_valid(X_train,Y_train,X_valid,Y_valid,fold)
    X_semi , Y_semi = get_semi_data(X_train,Y_train)
    print('===train semi_'+str(fold)+'===')
    model_semi = train_unverified(X_semi,Y_semi,fold)
# MFCC7
#6=>0.87062
#0=>0.87062 (X) 需要在一次fine tune
#1=>0.87332
#2=>0.87332
#3=>0.90296
#7=>0.87332
#4=>

/home/leoqaz12/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:32: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.


(3339, 60, 259, 1) (3339, 41)
(8162, 60, 259, 1) (8162, 41)
===train semi_4===
semi loading: model/mfcc7/LGD_fold4_co_resnet.h5
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_10 (InputLayer)           (None, 60, 259, 1)   0                                            
__________________________________________________________________________________________________
conv2d_582 (Conv2D)             (None, 30, 130, 64)  3200        input_10[0][0]                   
__________________________________________________________________________________________________
batch_normalization_559 (BatchN (None, 30, 130, 64)  256         conv2d_582[0][0]                 
__________________________________________________________________________________________________
activation_550 (Activation)     (None, 30, 130, 64)  0           batch_normaliza

Epoch 1/3000
510/510 [==============================] - 144s 283ms/step - loss: 1.5820 - acc: 0.7837 - val_loss: 1.0387 - val_acc: 0.7871

Epoch 00001: val_acc improved from -inf to 0.78706, saving model to model/mfcc7/LGD_fold4_self_resnet.h5
Epoch 2/3000
510/510 [==============================] - 125s 245ms/step - loss: 1.5668 - acc: 0.7854 - val_loss: 1.0589 - val_acc: 0.7736

Epoch 00002: val_acc did not improve from 0.78706
Epoch 3/3000
510/510 [==============================] - 126s 246ms/step - loss: 1.5513 - acc: 0.7915 - val_loss: 1.0237 - val_acc: 0.7925

Epoch 00003: val_acc improved from 0.78706 to 0.79245, saving model to model/mfcc7/LGD_fold4_self_resnet.h5
Epoch 4/3000
510/510 [==============================] - 126s 246ms/step - loss: 1.5502 - acc: 0.7918 - val_loss: 1.0181 - val_acc: 0.7763

Epoch 00004: val_acc did not improve from 0.79245
Epoch 5/3000
510/510 [==============================] - 126s 247ms/step - loss: 1.5493 - acc: 0.7887 - val_loss: 1.0083 - val_acc: 


Epoch 00040: val_acc did not improve from 0.83019
Epoch 41/3000
510/510 [==============================] - 126s 247ms/step - loss: 1.4247 - acc: 0.8143 - val_loss: 0.8840 - val_acc: 0.8059

Epoch 00041: val_acc did not improve from 0.83019
Epoch 42/3000
510/510 [==============================] - 126s 247ms/step - loss: 1.4015 - acc: 0.8227 - val_loss: 0.8144 - val_acc: 0.8410

Epoch 00042: val_acc improved from 0.83019 to 0.84097, saving model to model/mfcc7/LGD_fold4_self_resnet.h5
Epoch 43/3000
510/510 [==============================] - 126s 247ms/step - loss: 1.4019 - acc: 0.8224 - val_loss: 0.9323 - val_acc: 0.8059

Epoch 00043: val_acc did not improve from 0.84097
Epoch 44/3000
510/510 [==============================] - 126s 247ms/step - loss: 1.4075 - acc: 0.8202 - val_loss: 0.8899 - val_acc: 0.8032

Epoch 00044: val_acc did not improve from 0.84097
Epoch 45/3000
510/510 [==============================] - 126s 247ms/step - loss: 1.3868 - acc: 0.8296 - val_loss: 0.8271 - val_acc:

KeyboardInterrupt: 

In [8]:
for fold in val_set_num:
    X, y = getTrainData()
    # X = np.swapaxes(X,2,3)
    X_train, Y_train, X_valid, Y_valid = split_data(X, y, fold) #fold
    # X_train, X_valid = normalize(X_train, X_valid)
    print(X_train.shape, Y_train.shape)

    # X_train = np.swapaxes(X_train,1,3)
    # X_valid = np.swapaxes(X_valid,1,3)
#     print("===train verified_fold"+str(fold)+'_'+feature_type+'===')
#     model,model_num = train_valid(X_train,Y_train,X_valid,Y_valid,fold)
    X_semi , Y_semi = get_semi_data(X_train,Y_train)
    print('===train semi_'+str(fold)+'===')
    model_semi = train_unverified(X_semi,Y_semi,fold)

#MFCC6
#0=>0.83288
#1=>0.89757
#2=>0.8625
#3=>0.83288 (X)重頭train，重新load在tune
#4=>0.89218
#5=> 0.26146重頭train(X)設定大一點的lr
#6=>0.88679
#7=>0.89218
#8=>0.8814 (patience太小)需要在fine tune
#9=>0.92453

/home/leoqaz12/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:32: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.


(3339, 64, 431, 1) (3339, 41)
(8162, 64, 431, 1) (8162, 41)
===train semi_6===
semi loading: model/mfcc6/LGD_fold6_co_resnet.h5
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            (None, 64, 431, 1)   0                                            
__________________________________________________________________________________________________
conv2d_214 (Conv2D)             (None, 32, 216, 64)  3200        input_5[0][0]                    
__________________________________________________________________________________________________
batch_normalization_204 (BatchN (None, 32, 216, 64)  256         conv2d_214[0][0]                 
__________________________________________________________________________________________________
activation_200 (Activation)     (None, 32, 216, 64)  0           batch_normaliza

Epoch 1/3000
510/510 [==============================] - 181s 355ms/step - loss: 1.4374 - acc: 0.8197 - val_loss: 0.7829 - val_acc: 0.8518

Epoch 00001: val_acc improved from -inf to 0.85175, saving model to model/mfcc6/LGD_fold6_self_resnet.h5
Epoch 2/3000
510/510 [==============================] - 162s 317ms/step - loss: 1.4192 - acc: 0.8301 - val_loss: 0.8296 - val_acc: 0.8383

Epoch 00002: val_acc did not improve from 0.85175
Epoch 3/3000
510/510 [==============================] - 162s 318ms/step - loss: 1.4061 - acc: 0.8274 - val_loss: 0.7856 - val_acc: 0.8518

Epoch 00003: val_acc improved from 0.85175 to 0.85175, saving model to model/mfcc6/LGD_fold6_self_resnet.h5
Epoch 4/3000
510/510 [==============================] - 162s 318ms/step - loss: 1.4015 - acc: 0.8269 - val_loss: 0.7647 - val_acc: 0.8518

Epoch 00004: val_acc did not improve from 0.85175
Epoch 5/3000
510/510 [==============================] - 162s 318ms/step - loss: 1.3901 - acc: 0.8307 - val_loss: 0.7758 - val_acc: 

510/510 [==============================] - 163s 319ms/step - loss: 1.3091 - acc: 0.8412 - val_loss: 0.7326 - val_acc: 0.8544

Epoch 00041: val_acc did not improve from 0.87601
Epoch 42/3000
510/510 [==============================] - 163s 319ms/step - loss: 1.3162 - acc: 0.8379 - val_loss: 0.7538 - val_acc: 0.8518

Epoch 00042: val_acc did not improve from 0.87601
Epoch 43/3000
510/510 [==============================] - 163s 319ms/step - loss: 1.3102 - acc: 0.8433 - val_loss: 0.7590 - val_acc: 0.8652

Epoch 00043: val_acc did not improve from 0.87601
Epoch 44/3000
510/510 [==============================] - 163s 319ms/step - loss: 1.3013 - acc: 0.8437 - val_loss: 0.7374 - val_acc: 0.8598

Epoch 00044: val_acc did not improve from 0.87601
Epoch 45/3000
510/510 [==============================] - 163s 319ms/step - loss: 1.2971 - acc: 0.8455 - val_loss: 0.7104 - val_acc: 0.8679

Epoch 00045: val_acc did not improve from 0.87601
Epoch 46/3000
510/510 [==============================] - 163s 31

510/510 [==============================] - 162s 318ms/step - loss: 1.2254 - acc: 0.8536 - val_loss: 0.7152 - val_acc: 0.8706

Epoch 00083: val_acc did not improve from 0.88140
Epoch 84/3000
510/510 [==============================] - 162s 318ms/step - loss: 1.2286 - acc: 0.8502 - val_loss: 0.6867 - val_acc: 0.8787

Epoch 00084: val_acc did not improve from 0.88140
Epoch 85/3000
510/510 [==============================] - 162s 318ms/step - loss: 1.2424 - acc: 0.8508 - val_loss: 0.7177 - val_acc: 0.8706

Epoch 00085: val_acc did not improve from 0.88140
Epoch 86/3000
510/510 [==============================] - 162s 318ms/step - loss: 1.2329 - acc: 0.8529 - val_loss: 0.6589 - val_acc: 0.8868

Epoch 00086: val_acc improved from 0.88140 to 0.88679, saving model to model/mfcc6/LGD_fold6_self_resnet.h5
Epoch 87/3000
510/510 [==============================] - 162s 318ms/step - loss: 1.2227 - acc: 0.8543 - val_loss: 0.7074 - val_acc: 0.8652

Epoch 00087: val_acc did not improve from 0.88679
Epoch 

...


Epoch 00042: val_acc did not improve from 0.88140
Epoch 43/3000
510/510 [==============================] - 73s 143ms/step - loss: 0.9842 - acc: 0.8917 - val_loss: 0.7940 - val_acc: 0.8706

Epoch 00043: val_acc did not improve from 0.88140
Epoch 44/3000
510/510 [==============================] - 73s 143ms/step - loss: 0.9838 - acc: 0.8912 - val_loss: 0.7027 - val_acc: 0.8733

Epoch 00044: val_acc did not improve from 0.88140
Epoch 45/3000
510/510 [==============================] - 73s 143ms/step - loss: 0.9916 - acc: 0.8931 - val_loss: 0.7129 - val_acc: 0.8868

Epoch 00045: val_acc improved from 0.88140 to 0.88679, saving model to model/mfcc6/LGD_fold9_self_resnet.h5
Epoch 46/3000
510/510 [==============================] - 73s 143ms/step - loss: 0.9882 - acc: 0.8869 - val_loss: 0.7848 - val_acc: 0.8625

Epoch 00046: val_acc did not improve from 0.88679
Epoch 47/3000
510/510 [==============================] - 73s 143ms/step - loss: 0.9892 - acc: 0.8918 - val_loss: 0.6931 - val_acc: 0.87


Epoch 00084: val_acc improved from 0.89488 to 0.90027, saving model to model/mfcc6/LGD_fold9_self_resnet.h5
Epoch 85/3000
510/510 [==============================] - 73s 143ms/step - loss: 0.9007 - acc: 0.9029 - val_loss: 0.6478 - val_acc: 0.8868

Epoch 00085: val_acc did not improve from 0.90027
Epoch 86/3000
510/510 [==============================] - 73s 143ms/step - loss: 0.8953 - acc: 0.9048 - val_loss: 0.6333 - val_acc: 0.8922

Epoch 00086: val_acc did not improve from 0.90027
Epoch 87/3000
510/510 [==============================] - 73s 143ms/step - loss: 0.9012 - acc: 0.9023 - val_loss: 0.6440 - val_acc: 0.8841

Epoch 00087: val_acc did not improve from 0.90027
Epoch 88/3000
510/510 [==============================] - 73s 143ms/step - loss: 0.8904 - acc: 0.9035 - val_loss: 0.6541 - val_acc: 0.8868

Epoch 00088: val_acc did not improve from 0.90027
Epoch 89/3000
510/510 [==============================] - 73s 143ms/step - loss: 0.8890 - acc: 0.9028 - val_loss: 0.6000 - val_acc: 0.90

510/510 [==============================] - 73s 143ms/step - loss: 0.8612 - acc: 0.9081 - val_loss: 0.6037 - val_acc: 0.8895

Epoch 00127: val_acc did not improve from 0.90566
Epoch 128/3000
510/510 [==============================] - 73s 143ms/step - loss: 0.8610 - acc: 0.9078 - val_loss: 0.5861 - val_acc: 0.9030

Epoch 00128: val_acc did not improve from 0.90566
Epoch 129/3000
510/510 [==============================] - 73s 143ms/step - loss: 0.8637 - acc: 0.9033 - val_loss: 0.5972 - val_acc: 0.8976

Epoch 00129: val_acc did not improve from 0.90566
Epoch 130/3000
510/510 [==============================] - 73s 143ms/step - loss: 0.8556 - acc: 0.9079 - val_loss: 0.5473 - val_acc: 0.9137

Epoch 00130: val_acc improved from 0.90566 to 0.91375, saving model to model/mfcc6/LGD_fold9_self_resnet.h5
Epoch 131/3000
510/510 [==============================] - 73s 143ms/step - loss: 0.8564 - acc: 0.9080 - val_loss: 0.5977 - val_acc: 0.8922

Epoch 00131: ReduceLROnPlateau reducing learning rate to 

510/510 [==============================] - 73s 143ms/step - loss: 0.8040 - acc: 0.9108 - val_loss: 0.5226 - val_acc: 0.9057

Epoch 00169: val_acc did not improve from 0.91644
Epoch 170/3000
510/510 [==============================] - 73s 143ms/step - loss: 0.8100 - acc: 0.9085 - val_loss: 0.5105 - val_acc: 0.9164

Epoch 00170: val_acc did not improve from 0.91644
Epoch 171/3000
510/510 [==============================] - 73s 143ms/step - loss: 0.8074 - acc: 0.9108 - val_loss: 0.5280 - val_acc: 0.9164

Epoch 00171: val_acc did not improve from 0.91644
Epoch 172/3000
510/510 [==============================] - 73s 143ms/step - loss: 0.8082 - acc: 0.9120 - val_loss: 0.5259 - val_acc: 0.9137

Epoch 00172: ReduceLROnPlateau reducing learning rate to 6.24999984211172e-06.

Epoch 00172: val_acc did not improve from 0.91644
Epoch 173/3000
510/510 [==============================] - 73s 143ms/step - loss: 0.8020 - acc: 0.9129 - val_loss: 0.5184 - val_acc: 0.9057

Epoch 00173: val_acc did not improv

510/510 [==============================] - 73s 143ms/step - loss: 0.7900 - acc: 0.9159 - val_loss: 0.5290 - val_acc: 0.9164

Epoch 00211: val_acc did not improve from 0.92453
Epoch 212/3000
510/510 [==============================] - 73s 143ms/step - loss: 0.7923 - acc: 0.9122 - val_loss: 0.5269 - val_acc: 0.9218

Epoch 00212: val_acc did not improve from 0.92453
Epoch 213/3000
510/510 [==============================] - 73s 143ms/step - loss: 0.7884 - acc: 0.9112 - val_loss: 0.5213 - val_acc: 0.9164

Epoch 00213: val_acc did not improve from 0.92453
Epoch 214/3000
510/510 [==============================] - 73s 143ms/step - loss: 0.7959 - acc: 0.9149 - val_loss: 0.5161 - val_acc: 0.9218

Epoch 00214: val_acc did not improve from 0.92453
Epoch 215/3000
510/510 [==============================] - 73s 143ms/step - loss: 0.7858 - acc: 0.9157 - val_loss: 0.5282 - val_acc: 0.9111

Epoch 00215: val_acc did not improve from 0.92453
Epoch 216/3000
510/510 [==============================] - 73s 143

In [ ]:
#MFCC7
#0=> 0.8814
#1=> 0.85714
#2=> 0.87062
#3=> 0.88949
#4=> 0.82749
#5=> 0.84906
#6=> 0.85445
#7=> 0.84906
#8=> 0.84906
#9=> 0.90566

#====================
#0=>0.84367(resnet3_semi)/0.83288(not semi)
#1=>0.84367(resnet3_semi)/0.81132(not semi)
#2=>0.84097(resnet2_semi)/0.81671(not semi)
#3=>0.88410(resnet1_semi)/0.84367
#4=>0.82210(resnet4_semi)/0.74663(not semi)
#5 => 0.82749(resnet4_semi)/0.77628
#6 => 0.85445(resnet2_semi)/0.78437
#7 => 0.82749(resnet3_semi)/0.76280
#8 => 0.83288(resnet3_semi)/0.78706
# 9=> 0.90566(resnet1_semi)/0.87332

# 0=>0.88140 (co)
# 1 => 0.85714 (co)
# 2=>0.87062 (co)
# 3=>0.88949 (co)
# 4=>0.82749 (co)
# 5=>0.84367 (co)
# 6=>0.84906(X) (semi)
# 7=>0.84906 (co)
# 8=>0.83019(X) (smi)
# 9=>0.90566 (co)

In [ ]:
#MFCC6
#0=> 0.81941
#1=> 0.8841
#2=> 0.83558
#3=> 0.85984
#4=> 0.84367
#5=> 0.85175
#6=> 0.86792
#7=> 0.88949
#8=> 0.8841
#9=> 0.87332

#===============
# 0=> 0.81941 (not semi)
# 1=>0.83019 (semi)
# 2=>0.81941 (semi)
# 3=>0.85984 (resnet1_not semi)、0.78437
# 4=>0.84367 (renet1_not semi)、0.81132
# 5=> 0.85175(resnet3_semi)、0.82749(not semi)
#6 => 0.85904(resnet4_semi)、0.79784(not semi)
# 7=>0.88949(resnet2_semi)、0.83558(not semi)
# 8=>0.83558(resnet1 not semi)
# 9=>0.86792(resnet2_semi)、0.8112(not semi)

# 0 => 0.81941
#1=>0.88410
#2=>0.83558
# 3=>0.84367 (X)
# 4+>0.86523 (X)
# 5=> (X) (前五個fold街從頭train)

# 3=>0.85445 (x)
# 4=>0.82110 (x)
# 5=>0.84636 (X)
# 6=>0.86792
# 7=>0.88949
# 8=>0.88410
# 9=>0.87332